In [1]:
from body_matrix import load
from body_matrix import infer
from body_matrix import process
from body_matrix import score
from body_matrix import measure

video, frame_counts, fps, sample_frame = load.video(
    "/Users/troydo42/Desktop/Body_Matrixes/samples/sample02.mp4", 
    -90, 
    1
)

keypoints_model, keypoints_transform = load.keypoints_model("cpu")
selected_box, keypoints = infer.detect_main_target(
    sample_frame, "cpu", 0.8, keypoints_model, keypoints_transform
)

selected_kps = process.keypoints_filter(
    [
        'left_shoulder','right_shoulder', 
        'left_hip', 'right_hip',  
        'left_elbow','right_elbow',
        'left_wrist', 'right_wrist',
        'left_ankle', 'right_ankle'], 
    keypoints
)

segment_model, segment_transform = load.segment_model("cpu")
mask, mask_image, bool_mask = infer.segment_selected_target(
    sample_frame, "cpu", selected_box, 0.9, segment_model, segment_transform
)

segment_area = process.segmentation_area(
    sample_frame, 
    bool_mask
)

ls = selected_kps['left_shoulder']
rs = selected_kps['right_shoulder']
lh = selected_kps['left_hip']
rh = selected_kps['right_hip']

alpha_shoulder, beta_shoulder = score.two_points_linear_constant(ls, rs)
alpha_hip, beta_hip = score.two_points_linear_constant(lh, rh)

shoulder_line_coordinates = score.find_segment_line(
    segment_area, 
    alpha_shoulder, 
    beta_shoulder
)

hip_line_coordinates = score.find_segment_line(
    segment_area, 
    alpha_hip, 
    beta_hip
)

Found  3  person in the frame
Distances:  [4.6946258544921875, 357.71270751953125, 328.79632568359375]
Areas:  [309102.5849272795, 7238.688030660152, 4190.567056313157]
FOCUSES:  [65841.79325632645, 20.23603824660022, 12.745176052684391]
Select keypoint index 0
Found  7  person in the frame
[384.28985595703125, 757.6371459960938, 687.038330078125, 1751.9239501953125]
[384.4972839355469, 760.8311767578125, 694.1134643554688, 1759.17236328125]
[868.923583984375, 872.0135498046875, 930.5077514648438, 992.1439819335938]
[384.4972839355469, 760.8311767578125, 694.1134643554688, 1759.17236328125]
[863.081787109375, 787.0054931640625, 1087.1761474609375, 962.7617797851562]
[384.4972839355469, 760.8311767578125, 694.1134643554688, 1759.17236328125]
[9.776000022888184, 858.37646484375, 154.00531005859375, 933.3391723632812]
[384.4972839355469, 760.8311767578125, 694.1134643554688, 1759.17236328125]
[681.4767456054688, 970.3965454101562, 845.8213500976562, 1075.564453125]
[384.4972839355469, 760

In [2]:
if ls[0] < rs[0]:
    print("BACK")
    shoulder_kps = {
        'left_shoulder': shoulder_line_coordinates[0],
        'right_shoulder': shoulder_line_coordinates[-1]
    }

elif ls[0] > rs[0]:
    print("FRONT")
    shoulder_kps = {
        'left_shoulder': shoulder_line_coordinates[-1],
        'right_shoulder': shoulder_line_coordinates[0]
    }
        
        
shoulder_kps

BACK


{'left_shoulder': [403, 955], 'right_shoulder': [642, 957]}

In [3]:
lw = selected_kps['left_wrist']
rw = selected_kps['right_wrist']

if lw[1] > lh[1]:
    precise_rh = hip_line_coordinates[-1]
    precise_lhX = 2 * middle_hip[0] - rh[0]
    precise_lhY = hip_alpha * precise_lhX + hip_beta
    precise_lh = [int(precise_lhX), int(precise_lhY)]
    hip_kps = {
        'left_hip': precise_lh,
        'right_hip': precise_rh
    }
    print("low left hand", hip_kps)
    
elif rw[1] > rh[1]:
    precise_lh = hip_line_coordinates[0]
    precise_rhX = 2 * middle_hip[0] - precise_lh[0]
    precise_rhY = hip_alpha * precise_rhX + hip_beta
    precise_rh = [int(precise_rhX), int(precise_rhY)] 
    
    hip_kps = {
        'left_hip': precise_lh,
        'right_hip': precise_rh
    }
    print("low right hand", hip_kps)
    
else:
    hip_kps = {
        'left_hip': hip_line_coordinates[0],
        'right_hip': hip_line_coordinates[-1]
    }
    print("both hand high", hip_kps)

both hand high {'left_hip': [429, 1241], 'right_hip': [651, 1244]}


In [5]:
hip_kps = process.find_hip_points(
    lh, rh, lw, rw, segment_area
)

both hand high {'left_hip': [429, 1241], 'right_hip': [651, 1244]}


In [ ]:
updated_kps = {}
updated_kps.update(shoulder_kps)
updated_kps.update(hip_kps)

In [ ]:
shoulder_length = measure.two_points_distance(
    updated_kps['left_shoulder'],
    updated_kps['right_shoulder']
)

hip_length = measure.two_points_distance(
    updated_kps['left_hip'],
    updated_kps['right_hip']
)

middle_shoulder = measure.find_middle_point(
    updated_kps['left_shoulder'],
    updated_kps['right_shoulder']
)

middle_hip = measure.find_middle_point(
    updated_kps['left_hip'],
    updated_kps['right_hip']
)

middle_ankle = measure.find_middle_point(
    selected_kps['left_ankle'],
    selected_kps['right_ankle']
)

back_length = measure.two_points_distance(
    middle_shoulder, middle_hip
)

leg_length = measure.two_points_distance(
    middle_hip, middle_ankle
)

score = score.SHA_score(
    shoulder_length, 
    hip_length, 
    back_length, 
    leg_length
)
score